In [7]:
import os
import glob
import textgrid
import sys
import pandas as pd
path           = './'
word_level     = path + 'word_level'
video          = path + 'video' 
audio          = path +'audio' 
documents      = path + 'documents', 
metadata       = path+ 'metadata', 
transcriptions = path +'transcriptions_annotations'

In [8]:
def get_all_textgrid_files(path):
    filenames = []
    for root, dirs, files in os.walk(path):
        for file in files:
            if file.endswith(".TextGrid"):
                filenames.append(os.path.join(root, file))
    return filenames
annotation = get_all_textgrid_files(transcriptions)
word_level_annotation = get_all_textgrid_files(word_level)

In [9]:
anno = annotation[-1]
print(anno)

./transcriptions_annotations/r2/r2.TextGrid


In [10]:
word_level = word_level_annotation[-1]
print(word_level)

./word_level/r2B_wordlevel.TextGrid


In [48]:
def search_word_level(path, minTime, maxTime, text):
    tg = textgrid.TextGrid.fromFile(path)
    annotation = []
    for index, tier in enumerate(tg):
        for interval in tier:
            if interval.minTime >= minTime and interval.maxTime <= maxTime and tier.name == "ORT-MAU" and interval.mark==text:
                annotation.append((tier.name, interval.minTime, interval.maxTime, interval.mark))
    return annotation

In [53]:
r2_annotation = textgrid.TextGrid.fromFile(anno)
A_utters = []
B_utters = []
for index, tier in enumerate(r2_annotation):
    for interval in tier:
        if '+' in interval.mark:
            marks = interval.mark.split('+')
            substring = ""
            list_of_dis = []
            for pos, char in enumerate(interval.mark):
                plus_index = -1
                if char == "+":
                    substring = interval.mark[pos:]
#                     print(f"subtring {substring}")
#                     print(substring.split(" "))
#                     print(interval.mark)
#                     print(tier.name, substring.split(" ")[1])
                    if tier.name == "A-utts":
                        A_utters.append(  (substring.split(" ")[1], interval.minTime, interval.maxTime)  )
                    if tier.name == "B-utts":
                        B_utters.append( (substring.split(" ")[1], interval.minTime, interval.maxTime)   )
print(A_utters)
print(B_utters)

[('.', 553.78297, 559.48042), (')', 573.71455, 577.8176), ("<v='eine'>'ne</v>", 596.57193, 599.58429), (')', 612.05256, 615.54967), ('(', 668.1045, 671.3852), ('zu', 668.1045, 671.3852), (')', 680.33738, 683.39303), ('', 698.01669, 700.90953), ('{F', 710.11274, 718.46764), ('/genau', 710.11274, 718.46764), ("(<p='wenn'>w-</p>", 768.19888, 773.00474), ('wenn', 768.19888, 773.00474), ('auf', 794.034, 799.8947), ("(<p='Wohnzimmer'>w-</p>+", 794.034, 799.8947), ('im', 794.034, 799.8947), ('...', 824.34604, 830.68405), ('dann', 848.0171, 852.75371), ('so', 882.68322, 887.26401), ('(', 898.79741, 903.23805), ('was', 898.79741, 903.23805), ('Wohnzimmer)', 910.51958, 914.30234), ('fünfzig', 915.63706, 916.93549), ("(<p='immer'>", 955.30688, 957.07275), ('', 955.30688, 957.07275), ("<v='eine'>'ne</v>", 1023.27309, 1026.64901), ('', 1107.02122, 1108.63128), (')', 1121.09963, 1124.18599), ('wenn', 1151.5674, 1159.35099), ('den', 1165.0571, 1168.00886), ('<laughter/>', 1171.14821, 1176.61219), ('s

In [64]:
print(len(A_utters))
word_level = word_level_annotation[-1]
print()

# A_word_level = []
# for utters in A_utters:
# #     print(utters)
#     text, minTime, maxTime = utters
#     if len(search_word_level('./word_level/r2A_wordlevel.TextGrid', minTime, maxTime, text)) > 0:
# #         print("--->", search_word_level('./word_level/r2A_wordlevel.TextGrid', minTime, maxTime, text))
#         A_word_level.extend(search_word_level('./word_level/r2A_wordlevel.TextGrid', minTime, maxTime, text))
# print(len(A_word_level))

41


In [65]:
print(len(B_utters))

27


In [63]:
# B_word_level = []
# for utters in B_utters:
# #     print(utters)
#     text, minTime, maxTime = utters
#     if len(search_word_level('./word_level/r2B_wordlevel.TextGrid', minTime, maxTime, text)) > 0:
# #         print("--->", search_word_level('./word_level/r2A_wordlevel.TextGrid', minTime, maxTime, text))
#         B_word_level.extend(search_word_level('./word_level/r2A_wordlevel.TextGrid', minTime, maxTime, text))
# print(len(B_word_level))

2
